<a href="https://colab.research.google.com/github/rainerpm/CodingBatScraper/blob/main/NOAAProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# started with code from https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859

import requests  #needed to make web requests
import pandas as pd #store the data we get as a dataframe
import json #convert the response as a strcuctured json
import numpy as np #mathematical operations on lists
from datetime import datetime  #parse the datetimes we get from NOAA

# with open('token.txt', 'r') as fp:
#   token = fp.read().strip()
token = "XQilQLcTnIxYLmIXmRpHjOnOdszCShQT"

#station_id = "GHCND:USW00023129"  # from example at https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
#station_id = "GHCND:USC00410433"  # Austin Great Hills Trail (DOESN'T WORK)
station_id = "GHCND:USW00013958"   # Camp Mabry https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00013958/detail
limit = 1000  # how many records

#initialize lists to store data
tempDates = []
windDates = []
tempValues = []
windValues = []


#for each year from 2015-2019 ...
for year in range(2015, 2020):
    year = str(year)
    print('working on year '+year)
    

    #make the api call
    datatypeid1 = "TMAX"  # Maximum Temperature
    datatypeid2 = "AWND" # Average wind speed
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid={datatypeid1}&datatypeid={datatypeid2}&limit={limit}&stationid={station_id}&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':token})
    #print(json.dumps(r.json(), indent=2))
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response 
    tmaxResults = [item for item in d['results'] if item['datatype']=='TMAX']
    windResults = [item for item in d['results'] if item['datatype']=='AWND']
    #get the date field 
    tempDates += [item['date'] for item in tmaxResults]
    windDates += [item['date'] for item in windResults]
    #get the actual average temperature from all average temperature readings
    tempValues += [item['value'] for item in tmaxResults]
    windValues += [item['value'] for item in windResults]


#initialize dataframe
df_temp = pd.DataFrame()
df_wind = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in tempDates]
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in tempValues]

df_wind['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in windDates]
df_wind['avgWind'] = windValues




working on year 2015
working on year 2016
working on year 2017
working on year 2018
working on year 2019


In [ ]:
# Datatypes
params = {
    'startdate' : '2015-01-01',
    'enddate' : '2019-12-31'
}
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes', params=params, headers={'token':token})
d = json.loads(r.text)
print(d)

{'metadata': {'resultset': {'offset': 1, 'count': 357, 'limit': 25}}, 'results': [{'mindate': '1991-06-05', 'maxdate': '2021-07-11', 'name': 'Base Data', 'datacoverage': 0.95, 'id': 'ALL'}, {'mindate': '1982-01-01', 'maxdate': '2021-07-11', 'name': 'Average wind speed', 'datacoverage': 1, 'id': 'AWND'}, {'mindate': '1763-01-01', 'maxdate': '2021-06-01', 'name': 'Cooling Degree Days Season to Date', 'datacoverage': 1, 'id': 'CDSD'}, {'mindate': '1763-01-01', 'maxdate': '2021-06-01', 'name': 'Cooling Degree Days', 'datacoverage': 1, 'id': 'CLDD'}, {'mindate': '2011-03-08', 'maxdate': '2021-07-11', 'name': 'Digital Accumulation Array(One Hour Precipitation total (256 levels) with Dual Pol. algorithm)', 'datacoverage': 0.95, 'id': 'DAA'}, {'mindate': '1832-05-11', 'maxdate': '2021-07-12', 'name': 'Number of days included in the multiday precipitation total (MDPR)', 'datacoverage': 1, 'id': 'DAPR'}, {'mindate': '1877-01-02', 'maxdate': '2020-12-18', 'name': 'Number of days included in the m

In [ ]:
# Fetch data types that support a given set of stations
params = {
    'limit'   : 999
}
r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?stationid={station_id}', params=params, headers={'token':token})
#print(json.dumps(r.json(), indent=2))
d = json.loads(r.text)
results = d["results"]
print(len(results))
for result in results:
  #if "AVG" in result["id"]:
     print(result["id"], result["name"])
print(d)

7
DP05 Number of days with greater than or equal to 0.5 inch of precipitation
DSNW Number days with snow depth > 1 inch.
DYSN Day the extreme maximum daily snowfall for the period occurred.
DYXP Day Extreme maximum daily precipitation for the period.
EMSN Extreme maximum snowfall for the period.
TPCP Total precipitation
TSNW Total snow fall
{'metadata': {'resultset': {'offset': 1, 'count': 7, 'limit': 999}}, 'results': [{'mindate': '1781-01-01', 'maxdate': '2016-03-01', 'name': 'Number of days with greater than or equal to 0.5 inch of precipitation', 'datacoverage': 1, 'id': 'DP05'}, {'mindate': '1840-05-01', 'maxdate': '2021-06-01', 'name': 'Number days with snow depth > 1 inch.', 'datacoverage': 1, 'id': 'DSNW'}, {'mindate': '1840-05-01', 'maxdate': '2021-06-01', 'name': 'Day the extreme maximum daily snowfall for the period occurred.', 'datacoverage': 1, 'id': 'DYSN'}, {'mindate': '1781-01-01', 'maxdate': '2021-06-01', 'name': 'Day Extreme maximum daily precipitation for the period.

In [10]:
df_temp.keys()
df_temp



,date,avgTemp
0,2015-01-01,39.92
1,2015-01-02,42.98
2,2015-01-03,62.06
3,2015-01-04,51.08
4,2015-01-05,51.98
...,...,...
1821,2019-12-27,68.00
1822,2019-12-28,71.96
1823,2019-12-29,66.02
1824,2019-12-30,62.06


In [11]:
df_wind.keys()
df_wind

,date,avgWind
0,2015-01-01,22
1,2015-01-02,20
2,2015-01-03,13
3,2015-01-04,32
4,2015-01-05,16
...,...,...
1821,2019-12-27,15
1822,2019-12-28,22
1823,2019-12-29,26
1824,2019-12-30,14
